# The Drivers of Spectral Variance

This notebook contains routines to simulate a series of vegetation spectra based on global variance in leaf and canopy structural parameters.

In [ ]:
# run these command only on the first time you run this notebook
!pip3 install pyprosail
#!cp /usr/local/lib/python3.6/dist-packages/PyPROSAIL/*.so /usr/local/lib/python3.6/dist-packages/pyprosail/
!cp /home/cba/src/miniconda3/envs/earthtools/lib/python3.7/site-packages/PyPROSAIL/*.so /home/cba/src/miniconda3/envs/earthtools/lib/python3.7/site-packages/pyprosail/

In [ ]:
# imports
import os
import random
import spectral
import pyprosail
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# setting paths
base = "/home/cba/src/grad-school/projects/dissertation"
plot = f"{base}/figures/"
if not os.path.exists(plot):
    os.mkdir(plot)

## Setting up the parameter space

In [ ]:
# set the number of spectra to simulate
n_spectra = 1000
nb_prosail = 2101
spectra_prosail = np.zeros((n_spectra, nb_prosail))

# create the arrays to store parameters in
N = np.zeros(n_spectra, dtype=float)
chloro = np.zeros(n_spectra, dtype=float)
caroten = np.zeros(n_spectra, dtype=float)
brown = np.zeros(n_spectra, dtype=float)
EWT = np.zeros(n_spectra, dtype=float)
LMA = np.zeros(n_spectra, dtype=float)
soil_reflectance = np.zeros(n_spectra, dtype=float)
LAI = np.zeros(n_spectra, dtype=float)
hot_spot = np.zeros(n_spectra, dtype=float)
LAD_inclination = np.zeros(n_spectra, dtype=float)
LAD_bimodality = np.zeros(n_spectra, dtype=float)
s_az = np.zeros(n_spectra, dtype=float)
s_za = np.zeros(n_spectra, dtype=float)
v_az = np.zeros(n_spectra, dtype=float)
v_za = np.zeros(n_spectra, dtype=float)

# use the global range of parameter space to sample random permutations of veg growth/structure
for i in range(n_spectra):
    
    # prosail structural coefficient (arbitrary units)
    #  range 1.3 - 2.5 from Rivera et al. 2013 http://dx.doi.org/10.3390/rs5073280
    N[i] = random.uniform(1.3,2.5)

    # total chlorophyll content (ug/cm^2)
    #  range ~ 5 - 75 from Rivera et al. 2013, but i'll set a little more conservative
    chloro[i] = random.gauss(35, 30)
    while chloro[i] < 10 or chloro[i] > 60:
        chloro[i] = random.gauss(35, 30)

    # total carotenoid content (ug/cm^2)
    #  kinda fudged this to be like 1/4 of total chl
    caroten[i] = random.gauss(8.75, 7.5)
    while caroten[i] < 2 or caroten[i] > 15:
        caroten[i] = random.gauss(8.75, 7.5)

    # brown pigment content (arbitrary units) - not gonna mess with this
    brown[i] = 0.

    # equivalent water thickness (cm)
    #  range 0.002 - 0.05 from Rivera et al. 2013
    EWT[i] = random.uniform(0.002, 0.05)

    # leaf mass per area (g/cm^2)
    #  global range 0.0022 - 0.0365 (median 0.01)
    #  from Asner et al. 2011 http://dx.doi.org/10.1016/j.rse.2011.08.020
    # gonna go a little more conservative
    LMA[i] = random.gauss(0.012, 0.005)
    while LMA[i] < 0.005 or LMA[i] > 0.0250:
        LMA[i] = random.gauss(0.012, 0.005)

    # soil reflectance metric (wet soil = 0, dry soil = 1)
    soil_reflectance[i] = random.uniform(0,1)

    # leaf area index (unitless, cm^2 leaf area/cm^2 ground area)
    #  range 0.01 - 18.0 (5.5 mean) globally
    #  range 0.2 - 8.7 (3.6 mean) for crops
    #  range 0.6 - 2.8 (1.3 mean) for desert plants
    #  range 0.5 - 6.2 (2.6 mean) for boreal broadleaf forest
    #  range 0.5 - 8.5 (4.6 mean) for boreal needle forest
    #  range 0.8 - 11.6 (5.1 mean) for temperate broadleaf forest
    #  range 0.01 - 15.0 (5.5 mean) for temperate needle forest
    #  range 0.6 - 8.9 (4.8 mean) for tropical broadleaf forest
    #  range 0.3 - 5.0 (1.7 mean) for grasslands
    #  range 1.6 - 18.0 (8.7 mean) for plantations
    #  range 0.4 - 4.5 (2.1 mean) for shrublands
    #  range 0.2 - 5.3 (1.9 mean) for tundra
    #  range 2.5 - 8.4 (6.3 mean) for wetlands
    #  from Asner, Scurlock and Hicke 2003 http://dx.doi.org/10.1046/j.1466-822X.2003.00026.x
    LAI[i] = random.gauss(3,2)
    while LAI[i] < 0.25 or LAI[i] > 15:
        LAI[i] = random.gauss(3,2)

    # hot spot parameter (derived from brdf model)
    #  range 0.05-0.5 from Rivera et al. 2013
    hot_spot[i] = random.uniform(0.05, 0.5)

    # leaf distribution function parameter.
    #  range LAD_inc -0.4 -  0.4, LAD_bim -0.1 - 0.2 for trees
    #  range LAD_inc -0.1 -  0.3, LAD_bim  0.3 - 0.5 for lianas
    #  range LAD_inc -0.8 - -0.2, LAD_bim -0.1 - 0.3 for palms
    #  from Asner et al. 2011
    LAD_inclination[i] = random.uniform(-0.4, 0.4)
    LAD_bimodality[i] = random.uniform(-0.1, 0.2)

    # viewing and solar angle parameters
    #  solar zenith ranges cludged from http://gis.stackexchange.com/questions/191692/maximum-solar-zenith-angle-for-landsat-8-images
    #  I couldn't find good data on the range of possible solar or viewing azimuth.
    #  I decided to set view parameters to 0 to assume nice, clean nadir viewing, and let the sun vary.
    s_za[i] = random.uniform(20, 70)
    s_az[i] = random.uniform(0,360)
    v_az[i] = 0.
    v_za[i] = 0.

In [ ]:
# print some of the resulting data to get a sense for the range and variance
metrics = [chloro, EWT, LMA, LAI]
names = ['CHL', 'EWT', 'LMA', 'LAI']
print("Metric | Mean | Min | Max")
for metric, name in zip(metrics, names):
    low = np.percentile(metric, 2.5)
    high = np.percentile(metric, 97.5)
    mean = metric.mean()
    print(f"{name} | {mean:0.3f} | {low:0.3f} | {high:0.3f}")

## Isolate and plot the spectral variance for each metric

In [ ]:
# set basic plot / etc. parameters
n = 5.
colors = ["#BFCE68", "#AAC03B", "#8FB926", "#67A612", "#477706"]
wv = np.arange(0.4, 2.501, 0.001)
figsize = (5,5)
dpi = 300

# filter water vapor wavelengths
swir1 = (wv > 1.30) * (wv < 1.45)
swir2 = (wv > 1.80) * (wv < 1.96)
bad_bands = swir1 + swir2

In [ ]:
# first, chlorophyll content
metric = 'Chlorophyll Content'
units = "$\mu g \cdot cm^{-2}$"
minv = 15
maxv = 60
by = (maxv-minv) / (n-1)
rangev = np.arange(minv, maxv+by, by)

# create the plot
plt.figure(figsize=figsize, dpi=dpi)

# simulate each spectrum
for index, chl in enumerate(rangev):
    #LIDF = (LAD_inclination.mean(), LAD_biomodality.mean())
    LIDF = pyprosail.Planophile
    spectrum = pyprosail.run(
        N.mean(),
        chl,
        caroten.mean(),
        brown.mean(),
        EWT.mean(),
        LMA.mean(),
        soil_reflectance.mean(),
        LAI.mean(),
        hot_spot.mean(),
        s_za.mean(),
        s_az.mean(),
        v_za.mean(),
        v_az.mean(),
        LIDF,
    )
    spectrum = spectrum[:, 1] * 100
    spectrum[bad_bands] = np.nan
    
    # plot it
    label = f"{int(chl):2d} {units}"
    plt.plot(wv, spectrum, c = colors[index], label=label)
    
# scale the ranges
plt.ylim(0, 70)
plt.title(f"{metric}", fontsize=15)
plt.xlabel("Wavelength ($\mu m$)", fontsize=12)
plt.ylabel("Reflectance ($\%$)", fontsize=12)
plt.legend(loc='upper right')
plt.tight_layout()

# save it up
fname = f"{plot}{metric} (labeled)"
plt.savefig(fname + '.svg')
plt.savefig(fname + '.jpg', dpi=dpi)

# and do it again without labels
plt.xlabel("")
plt.ylabel("")
plt.tight_layout()
fname = f"{plot}{metric} (unlabeled)"
plt.savefig(fname + '.svg')
plt.savefig(fname + '.jpg', dpi=dpi)

In [ ]:
# then EWT
metric = 'Leaf Water Thickness'
units = "$mm$"
minv = 0.005
maxv = 0.05
by = (maxv-minv) / (n-1)
rangev = np.arange(minv, maxv+by, by)

# create the plot
plt.figure(figsize=figsize, dpi=dpi)

# simulate each spectrum
for index, ewt in enumerate(rangev):
    #LIDF = (LAD_inclination.mean(), LAD_biomodality.mean())
    LIDF = pyprosail.Planophile
    spectrum = pyprosail.run(
        N.mean(),
        chloro.mean(),
        caroten.mean(),
        brown.mean(),
        ewt,
        LMA.mean(),
        soil_reflectance.mean(),
        LAI.mean(),
        hot_spot.mean(),
        s_za.mean(),
        s_az.mean(),
        v_za.mean(),
        v_az.mean(),
        LIDF,
    )
    spectrum = spectrum[:, 1] * 100
    spectrum[bad_bands] = np.nan
    
    # plot it
    label = f"{(ewt*10):0.2f} {units}"
    plt.plot(wv, spectrum, c = colors[index], label=label)
    
# scale the ranges
plt.ylim(0, 70)
plt.title(f"{metric}", fontsize=15)
plt.xlabel("Wavelength ($\mu m$)", fontsize=12)
plt.ylabel("Reflectance ($\%$)", fontsize=12)
plt.legend(loc='upper right')
plt.tight_layout()

# save it up
fname = f"{plot}{metric} (labeled)"
plt.savefig(fname + '.svg')
plt.savefig(fname + '.jpg', dpi=dpi)

# and do it again without labels
plt.xlabel("")
plt.ylabel("")
plt.tight_layout()
fname = f"{plot}{metric} (unlabeled)"
plt.savefig(fname + '.svg')
plt.savefig(fname + '.jpg', dpi=dpi)

In [ ]:
# then LMA
metric = 'Leaf Mass Per Area'
units = "$g \cdot m^{-2}$"
minv = 0.005
maxv = 0.02
by = (maxv-minv) / (n-1)
rangev = np.arange(minv, maxv+by, by)

# create the plot
plt.figure(figsize=figsize, dpi=dpi)

# simulate each spectrum
for index, lma in enumerate(rangev):
    #LIDF = (LAD_inclination.mean(), LAD_biomodality.mean())
    LIDF = pyprosail.Planophile
    spectrum = pyprosail.run(
        N.mean(),
        chloro.mean(),
        caroten.mean(),
        brown.mean(),
        EWT.mean(),
        lma,
        soil_reflectance.mean(),
        LAI.mean(),
        hot_spot.mean(),
        s_za.mean(),
        s_az.mean(),
        v_za.mean(),
        v_az.mean(),
        LIDF,
    )
    spectrum = spectrum[:, 1] * 100
    spectrum[bad_bands] = np.nan
    
    # plot it
    label = f"{int(lma * 10000):2d} {units}"
    plt.plot(wv, spectrum, c = colors[index], label=label)
    
# scale the ranges
plt.ylim(0, 70)
plt.title(f"{metric}", fontsize=15)
plt.xlabel("Wavelength ($\mu m$)", fontsize=12)
plt.ylabel("Reflectance ($\%$)", fontsize=12)
plt.legend(loc='upper right')
plt.tight_layout()

# save it up
fname = f"{plot}{metric} (labeled)"
plt.savefig(fname + '.svg')
plt.savefig(fname + '.jpg', dpi=dpi)

# and do it again without labels
plt.xlabel("")
plt.ylabel("")
plt.tight_layout()
fname = f"{plot}{metric} (unlabeled)"
plt.savefig(fname + '.svg')
plt.savefig(fname + '.jpg', dpi=dpi)

In [ ]:
# then LAI
metric = 'Leaf Area Index'
units = "$m^2 \cdot m^{-2}$"
minv = 1
maxv = 5
by = (maxv-minv) / (n-1)
rangev = np.arange(minv, maxv+by, by)

# create the plot
plt.figure(figsize=figsize, dpi=dpi)

# simulate each spectrum
for index, lai in enumerate(rangev):
    #LIDF = (LAD_inclination.mean(), LAD_biomodality.mean())
    LIDF = pyprosail.Planophile
    spectrum = pyprosail.run(
        N.mean(),
        chloro.mean(),
        caroten.mean(),
        brown.mean(),
        EWT.mean(),
        LMA.mean(),
        soil_reflectance.mean(),
        lai,
        hot_spot.mean(),
        s_za.mean(),
        s_az.mean(),
        v_za.mean(),
        v_az.mean(),
        LIDF,
    )
    spectrum = spectrum[:, 1] * 100
    spectrum[bad_bands] = np.nan
    
    # plot it
    label = f"{int(lai)} {units}"
    plt.plot(wv, spectrum, c = colors[index], label=label)
    
# scale the ranges
plt.ylim(0, 70)
plt.title(f"{metric}", fontsize=15)
plt.xlabel("Wavelength ($\mu m$)", fontsize=12)
plt.ylabel("Reflectance ($\%$)", fontsize=12)
plt.legend(loc='upper right')
plt.tight_layout()

# save it up
fname = f"{plot}{metric} (labeled)"
plt.savefig(fname + '.svg')
plt.savefig(fname + '.jpg', dpi=dpi)

# and do it again without labels
plt.xlabel("")
plt.ylabel("")
plt.tight_layout()
fname = f"{plot}{metric} (unlabeled)"
plt.savefig(fname + '.svg')
plt.savefig(fname + '.jpg', dpi=dpi)